In [13]:
import pandas as pd
import re
import emoji
import pandas as pd
import ast
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import random

In [ ]:
fact_checks = pd.read_csv("../dataset/fact_checks.csv")
posts = pd.read_csv("../dataset/posts.csv")
pairs = pd.read_csv("../dataset/pairs.csv")


In [15]:
def replace_text_with_ocr(df, text_column='text', ocr_column='ocr'):
    df[text_column] = df.apply(
        lambda row: row[ocr_column] if pd.isna(row[text_column]) or row[text_column].strip() == '' else row[text_column],
        axis=1
    )
    return df
import ast

import ast
import pandas as pd

def safe_literal_eval(val):
    try:
        if isinstance(val, str):
            parsed_val = ast.literal_eval(val)
            if isinstance(parsed_val, list):
                for item in parsed_val:
                    if isinstance(item, tuple) and len(item) == 3:
                        return item  # Return the first valid tuple
            elif isinstance(parsed_val, tuple) and len(parsed_val) == 3:
                return parsed_val
        return val
    except (ValueError, SyntaxError) as e:
        print(f"Literal eval failed for {val}: {e}")
        return (None, None, None)




def split_text_column(row, row_name):
    try:
        parsed = safe_literal_eval(row[row_name])
        if isinstance(parsed, tuple) and len(parsed) == 3:
            first_text = parsed[0].strip() if isinstance(parsed[0], str) else None
            second_text = parsed[1].strip() if isinstance(parsed[1], str) else None
            
            # Extract the language with the highest confidence
            if isinstance(parsed[2], list) and all(isinstance(item, tuple) for item in parsed[2]):
                lang_conf = max(parsed[2], key=lambda x: x[1] if len(x) == 2 else 0)
            else:
                lang_conf = (None, None)
            
            lang = lang_conf[0]
            confidence = lang_conf[1]
            return pd.Series([first_text, second_text, lang, confidence])
    except Exception as e:
        print(f"Error processing row: {row[row_name]} -> {e}")
    return pd.Series([None, None, None, None])

In [16]:
posts = replace_text_with_ocr(posts, text_column='text', ocr_column='ocr')
posts[['ocr_original', 'ocr_translated', 'ocr_language', 'ocr_confidence']] = posts.apply(
    lambda row: split_text_column(row, 'ocr'), axis=1
)

posts[['text_original', 'text_translated', 'text_language', 'text_confidence']] = posts.apply(
    lambda row: split_text_column(row, 'text'), axis=1
)


fact_checks[['original_claim', 'translated_claim', 'language', 'confidence']] = fact_checks.apply(
    lambda row: split_text_column(row, 'claim'), axis=1
)

In [ ]:
def create_pos_neg_pairs(train_df, fact_checks):
    pos_pairs = []
    for _, row in train_df.iterrows():
        pair_dict = row.to_dict()
        pair_dict['label'] = 1  # Positive label
        pos_pairs.append(pair_dict)
    
    print(f"Total Positive Pairs: {len(pos_pairs)}")

    fact_check_dict = fact_checks.set_index('fact_check_id').to_dict('index')

    neg_pairs = []
    for _, row in train_df.iterrows():
        post_text = row['text']
        correct_fact_check_id = row['fact_check_id']

        while True:
            random_fact_id = random.choice(list(fact_check_dict.keys()))
            if random_fact_id != correct_fact_check_id:
                break

        random_fact_check_data = fact_check_dict[random_fact_id]

        if pd.notna(post_text) and pd.notna(random_fact_check_data['claim']):
            neg_pairs.append({ 
                'ocr_original': row['ocr_original'], 
                'ocr_translated': row['ocr_translated'], 
                'text_original': row['text_original'], 
                'text_translated': row['text_translated'],
                'original_claim': random_fact_check_data['original_claim'], 
                'translated_claim': random_fact_check_data['translated_claim'],
                'label': 0  
            })

    print(f"Total Negative Samples: {len(neg_pairs)}")
    all_pairs = pos_pairs + neg_pairs
    df = pd.DataFrame(all_pairs)
    return df


def merge_data(posts, fact_checks, pairs):

    posts = posts.drop_duplicates(subset='post_id')
    fact_checks = fact_checks.drop_duplicates(subset='fact_check_id')
    
    merged_data = pairs.merge(posts, on='post_id', how='left').merge(fact_checks, on='fact_check_id', how='left')
    
    merged_data.drop(columns=['instances_x', 'verdicts', 'ocr_confidence', 'instances_y', 'confidence'], inplace=True)
    
    return merged_data

In [18]:
merged_data = merge_data(posts, fact_checks, pairs)


train_df, temp_df = train_test_split(merged_data, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_pairs = create_pos_neg_pairs(train_df, fact_checks)
test_pairs = create_pos_neg_pairs(test_df, fact_checks)
val_pairs = create_pos_neg_pairs(val_df, fact_checks)

   post_id                                                ocr  \
0     2228  [('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...   
1     2228  [('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...   
2     2228  [('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...   
3     2229  [('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...   
4     2229  [('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...   
5     2229  [('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...   
6     6088  [('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...   
7     6088  [('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...   
8     6088  [('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...   
9    21033                                                 []   

                                                text  
0  [('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...  
1  [('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...  
2  [('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...  
3  [('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...  
4  [('WHY

In [19]:
merged_data.head(2)

,post_id,fact_check_id,ocr,text,ocr_original,ocr_translated,ocr_language,text_original,text_translated,text_language,text_confidence,claim,title,original_claim,translated_claim,language
0,2228,33,"[('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...","[('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...","WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...","WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,"WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...","WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,1.0,"('""$4 trillion jobs plan"" unnecessary because ...",('Posts on Biden jobs plan falsely claim 2020 ...,"""$4 trillion jobs plan"" unnecessary because 20...","""$4 trillion jobs plan"" unnecessary because 20...",eng
1,2228,23568,"[('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...","[('WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHE...","WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...","WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,"WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...","WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,1.0,('America had the lowest unemployment rate in ...,('Fact check: Unemployment rate hit historic h...,America had the lowest unemployment rate in hi...,America had the lowest unemployment rate in hi...,eng


In [23]:
train_pairs.head(3)

,post_id,fact_check_id,ocr,text,ocr_original,ocr_translated,ocr_language,text_original,text_translated,text_language,text_confidence,claim,title,original_claim,translated_claim,language,label
0,10719.0,84382.0,[],('Fue en 1908. Los belgas leyendo la Biblia an...,None,None,None,Fue en 1908. Los belgas leyendo la Biblia ante...,It was in 1908. Belgians reading the Bible bef...,spa,1.0,('Los belgas leyendo la Biblia antes de colgar...,"('Según historiadores, es improbable que la fo...",Los belgas leyendo la Biblia antes de colgar a...,Belgians reading the Bible before hanging a 7-...,spa,1
1,21478.0,56588.0,[('TN Todo Noticias [USER] Noticias ULTIMO MOM...,"('Y seguimos sumando muertes... [URL]', 'And w...",TN Todo Noticias [USER] Noticias ULTIMO MOMENT...,TN All News [USER] News LAST MOMENT The journa...,spa,Y seguimos sumando muertes... [URL],And we keep adding deaths... [URL],spa,1.0,"('Falleció el periodista Carlos Ferrara, que h...",('El periodista argentino Carlos Ferrara no mu...,"Falleció el periodista Carlos Ferrara, que hab...","Journalist Carlos Ferrara died, who had fainte...",spa,1
2,14336.0,103924.0,"[('ان شهادت ه امام من شریک تمبوتراب اسکاوٹ', '...",('Masjid to carona ha lakin yha nae ha carona ...,ان شهادت ه امام من شریک تمبوتراب اسکاوٹ,that testimony My Imam Tembutrab Scout partner,urd,Masjid to carona ha lakin yha nae ha carona ku...,Masjid to carona ha lakin yha nae ha carona ku...,hin,1.0,"('Photo shows Syed Murad Ali Shah, chief minis...",('This photo shows a Pakistani provincial chie...,"Photo shows Syed Murad Ali Shah, chief ministe...","Photo shows Syed Murad Ali Shah, chief ministe...",eng,1
